# Introduction
The objective is to get a SSD network and apply the needed changes to use it on a Edge TPU. I am sharing this notebook because, even I think I am doing the right steps, in the end it doesn't work.

# Set-up
This step is to clone the repository. If you have done it once before, you can omit this step.

In [ ]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

## Imports
Needed step: This is just for making the imports

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import pathlib
import os
import random
import io
import imageio
import tensorflow as tf
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage


from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
#from object_detection.utils import colab_utils
from object_detection.utils import config_util
from object_detection.builders import model_builder

%matplotlib inline

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


## Downloading a friendly model
For tflite is recommended to use SSD networks.
I have downloaded the following model, it is about "object detection". It works with 320x320 images.

In [ ]:
# Download the checkpoint and put it into models/research/object_detection/test_data/

!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
!tar -xf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
!if [ -d "models/research/object_detection/test_data/checkpoint" ]; then rm -Rf models/research/object_detection/test_data/checkpoint; fi
!mkdir models/research/object_detection/test_data/checkpoint
!mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint models/research/object_detection/test_data/

In [2]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = '/home/jose/codeWorkspace-2.4.1/tf_2.4.1/models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# Export and run with TFLite

## Model conversion
On this step I convert the pb saved model to .tflite

In [ ]:
!tflite_convert --saved_model_dir=/home/jose/codeWorkspace-2.4.1/tf_2.4.1/tflite/saved_model --output_file=/home/jose/codeWorkspace-2.4.1/tf_2.4.1/tflite/model.tflite

## Model Quantization (From float to uint8)
Once the model is converted, I need to quantize it. The original model picks up a float as tensor input. As I want to run it on an Edge TPU I need the input and output tensors to be uint8.

In [3]:
## Generating a calibration data set. 
def representative_dataset_gen():
    folder = "/home/jose/codeWorkspace-2.4.1/tf_2.4.1/images_ssd_mb2_2"
    image_size = 320
    raw_test_data = []

    files = glob.glob(folder+'/*.jpeg')
    for file in files:
        image = Image.open(file)
        image = image.convert("RGB")
        image = image.resize((image_size, image_size))
        #Quantizing the image between -1,1;
        image = (2.0 / 255.0) * np.float32(image) - 1.0
        #image = np.asarray(image).astype(np.float32)
        image = image[np.newaxis,:,:,:]
        raw_test_data.append(image)

    for data in raw_test_data:
        yield [data]

### (DO NOT RUN THIS ONE). It is the above step but with random values
If you don't have a dataset, you also can introduce random generated values, as if it was an image. This is the code I used to do so:

In [ ]:
####THIS IS A RANDOM-GENERATED DATASET#### 
def representative_dataset_gen():
    for _ in range(320):
      data = np.random.rand(1, 320, 320, 3)
      yield [data.astype(np.float32)]

## Call for model convert

In [4]:
converter = tf.lite.TFLiteConverter.from_saved_model('/home/jose/codeWorkspace-2.4.1/tf_2.4.1/tflite/saved_model')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8, tf.lite.OpsSet.SELECT_TF_OPS]
#converter.experimental_new_converter = True
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
converter.allow_custom_ops = True
converter.representative_dataset = representative_dataset_gen
tflite_model = converter.convert()

### WARNING!!!
The conversion step returns a warning.

WARNING:absl:For model inputs containing unsupported operations which cannot be quantized, the `inference_input_type` attribute will default to the original type.

WARNING:absl:For model outputs containing unsupported operations which cannot be quantized, the `inference_output_type` attribute will default to the original type.

This makes me think conversion is not correct.

# Experimental Area @@@@@@@@@@@@@@@@@@@@@@@@

This area is for new trials. Is not part of the final work

In [ ]:
###IGNORE THIS CELL###  RETURNS THE SAME WARNING AS ABOVE.

# Integer Quantization - Input/Output=uint8
converter = tf.lite.TFLiteConverter.from_saved_model('/home/jose/codeWorkspace-2.4.1/tf_2.4.1/tflite/saved_model')
converter.experimental_new_converter = True
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_model = converter.convert()

In [ ]:
frozen_graph_file = '/home/jose/codeWorkspace-2.4.1/tf_2.4.1/tflite/saved_model/saved_model.pb'
input_arrays = ["normalized_input_image_tensor"] 
output_arrays = ['TFLite_Detection_PostProcess',
                 'TFLite_Detection_PostProcess:1',
                 'TFLite_Detection_PostProcess:2',
                 'TFLite_Detection_PostProcess:3']
input_shapes = {"normalized_input_image_tensor" : [1, 320, 320, 3]}
converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
    frozen_graph_file,
    input_arrays,output_arrays,
    input_shapes) 
converter.allow_custom_ops = True 
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE] 
converter.representative_dataset = _representative_dataset_gen 
tflite_model_quant = converter.convert() 
with open(tflite_model_quant_file, "wb") as tflite_file: 
    tflite_file.write(tflite_model_quant)

# End of Experimental Area @@@@@@@@@@@@@@@@@@

## Saving the model

In [5]:
with open('/home/jose/codeWorkspace-2.4.1/tf_2.4.1/tflite/model_full_integer_quant_wImage6.tflite'.format('/home/jose/codeWorkspace-2.4.1/tf_2.4.1/tflite/saved_model'), 'wb') as w:
    w.write(tflite_model)
print("tflite convert complete! - {}/home/jose/codeWorkspace-2.4.1/tf_2.4.1/tflite/model_full_integer_quant_wImage6.tflite".format('/home/jose/codeWorkspace-2.4.1/tf_2.4.1/tflite/saved_model'))

tflite convert complete! - /home/jose/codeWorkspace-2.4.1/tf_2.4.1/tflite/saved_model/home/jose/codeWorkspace-2.4.1/tf_2.4.1/tflite/model_full_integer_quant_wImage6.tflite


# Tests

## Test 1: Get TensorFlow version
I readed that it is recommended to use nightly for this. So in my case, version is 2.6.0

In [ ]:
print(tf.version.VERSION)

## Test 2: Get input/output tensor details

In [ ]:
interpreter = tf.lite.Interpreter(model_path="/home/jose/codeWorkspace-2.4.1/tf_2.4.1/tflite/model_full_integer_quant_wImage6.tflite")
interpreter.allocate_tensors()

print(interpreter.get_input_details())
print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
print(interpreter.get_output_details())

### Test 2 Results:

I get the following info:

[{'name': 'serving_default_input:0', 'index': 0, 'shape': array([  1, 320, 320,   3], dtype=int32), 'shape_signature': array([  1, 320, 320,   3], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.007843137718737125, 127), 'quantization_parameters': {'scales': array([0.00784314], dtype=float32), 'zero_points': array([127], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

[{'name': 'StatefulPartitionedCall:31', 'index': 377, 'shape': array([ 1, 10,  4], dtype=int32), 'shape_signature': array([ 1, 10,  4], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'StatefulPartitionedCall:32', 'index': 378, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([ 1, 10], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'StatefulPartitionedCall:33', 'index': 379, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([ 1, 10], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'StatefulPartitionedCall:34', 'index': 380, 'shape': array([1], dtype=int32), 'shape_signature': array([1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]

So, I think it is not quantizing it right. But in the following step, it seems to work.

## Test 3: Real Application. Does the generated TFLite model work?
In this step I just want to test if the object detector works. This script acts like a program which picks up a picture, a model and runs the inference. At the end, the script shows the execution time, the objects detected and it draws a red square on the detected objects, showing the photo on the image reader.

In [90]:
import argparse
import numpy as np
import cv2
import tensorflow.lite as tflite

from datetime import datetime
from PIL import Image
from PIL import ImageDraw
from PIL import ImageColor


def load_labels(filename):
  with open(filename, 'r') as f:
    return [line.strip() for line in f.readlines()]


interpreter = tflite.Interpreter('/home/jose/codeWorkspace-2.4.1/tf_2.4.1/tflite/model_full_integer_quant_wImage6.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
#print(input_details)
output_details = interpreter.get_output_details()
#print(output_details)

# check the type of the input and output tensors
floating_modelI = input_details[0]['dtype'] == np.uint8
floating_modelO = output_details[0]['dtype'] == np.uint8
#Result: Yes. Input and output are both uint8.


# NxHxWxC, H:1, W:2
height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]
img = Image.open('/home/jose/codeWorkspace-2.4.1/tf_2.4.1/images_ssd_mb2_2/out7.jpeg')#.resize((320, 320))

# add N dim
input_data = np.expand_dims(img, axis=0)
#Test if the array has the expected dimmensions (1, 320, 320, 3)
print(input_data.shape)



#input_data = (2.0 / 255.0) * np.float32(input_data) - 1.0
interpreter.set_tensor(input_details[0]['index'], input_data)

# ignore the 1st invoke
startTime = datetime.now()
interpreter.invoke()
delta = datetime.now() - startTime
print("Warm-up time:", '%.1f' % (delta.total_seconds() * 1000), "ms\n")


startTime = datetime.now()
interpreter.invoke()
delta = datetime.now() - startTime
print("Inference time:", '%.1f' % (delta.total_seconds() * 1000), "ms\n")

output_data = interpreter.get_tensor(output_details[0]['index'])

boxes = interpreter.get_tensor(output_details[0]['index'])
classes = interpreter.get_tensor(output_details[1]['index'])
scores = interpreter.get_tensor(output_details[2]['index'])
num = interpreter.get_tensor(output_details[3]['index'])

draw = ImageDraw.Draw(img)
im_w, im_h = img.size


i = 0
while  (scores[0][i]/255) > 0.5:
  (ymin, xmin, ymax, xmax) = boxes[0][i]
  #(left, right, top, bottom) =(xmin*im_w, xmax*im_w, ymin*im_h, ymax*im_h)
  #(left, right, top, bottom) =(xmin, xmax, ymin, ymax)
  (left, right, top, bottom) =(xmin/255, xmax/255, ymin/255, ymax/255)
  (left, right, top, bottom) =(left*320, right*320, top*320, bottom*320)
  #(left, right, top, bottom) =(left+16, right+16, top, bottom)
  print(left, right, top, bottom)
  #(left, right, top, bottom) =(xmin*1.255, xmax*1.255, ymin*1.255, ymax*1.255)
  draw.line([(left,top),(left,bottom),(right,bottom),(right,top),(left,top)],
  width = 5, fill = 'red')
  print([(left,top),(left,bottom),(right,bottom),(right,top),(left,top)])
  #print(im_w)
  #print(xmin)
  i = i + 1
  #Anado el break porque si no me hace un outbounds.
  if i == 10:
    break
  print(scores[0][i]/255)
  
img.show()
#cv_img = cv2.cvtColor(np.asarray(img), cv2.COLOR_RGB2BGR) 
#cv2.imshow('image',cv_img)
#cv2.waitKey(0)

print("Number of objects found:","%d\n" % i)



print("Finished\n")



(1, 320, 320, 3)
Warm-up time: 1899.7 ms

Inference time: 1876.4 ms

104.15686274509804 185.72549019607845 100.3921568627451 205.80392156862746
[(104.15686274509804, 100.3921568627451), (104.15686274509804, 205.80392156862746), (185.72549019607845, 205.80392156862746), (185.72549019607845, 100.3921568627451), (104.15686274509804, 100.3921568627451)]
0.7215686274509804
254.74509803921566 298.6666666666667 242.19607843137254 276.07843137254906
[(254.74509803921566, 242.19607843137254), (254.74509803921566, 276.07843137254906), (298.6666666666667, 276.07843137254906), (298.6666666666667, 242.19607843137254), (254.74509803921566, 242.19607843137254)]
0.6941176470588235
185.72549019607845 227.1372549019608 215.843137254902 249.72549019607843
[(185.72549019607845, 215.843137254902), (185.72549019607845, 249.72549019607843), (227.1372549019608, 249.72549019607843), (227.1372549019608, 215.843137254902), (185.72549019607845, 215.843137254902)]
0.6313725490196078
79.05882352941177 128.0 203.294

In [26]:
boxes

array([[[137,  88, 195, 134],
        [102, 140, 128, 172],
        [ 14,  18, 115, 163],
        [164,  36, 216,  57],
        [ 94,  21, 120,  55],
        [ 16,  28,  87, 153],
        [108,  58, 134,  72],
        [ 81,  98, 104, 120],
        [158, 146, 228, 237],
        [ 16,  16, 233, 233]]], dtype=uint8)

In [ ]:
#Abrir una imagen en el S.O.
imgPath = '/home/jose/python-envs/tf_2.4.1/images_ssd_mb2/out1.jpeg'
img = Image.open(imgPath)
img.show()

# Converting the generated model to EdgeTPU

In [ ]:
!edgetpu_compiler -s /home/jose/codeWorkspace-2.4.1/tf_2.4.1/tflite/model_full_integer_quant.tflite

jose@jose-VirtualBox:~/python-envs$ edgetpu_compiler -s /home/jose/codeWorkspace-2.4.1/tf_2.4.1/tflite/model_full_integer_quant.tflite
Edge TPU Compiler version 15.0.340273435

Model compiled successfully in 1136 ms.

Input model: /home/jose/codeWorkspace-2.4.1/tf_2.4.1/tflite/model_full_integer_quant.tflite
Input size: 3.70MiB
Output model: model_full_integer_quant_edgetpu.tflite
Output size: 4.21MiB
On-chip memory used for caching model parameters: 3.42MiB
On-chip memory remaining for caching model parameters: 4.31MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 162
Operation log: model_full_integer_quant_edgetpu.log

Model successfully compiled but not all operations are supported by the Edge TPU. A percentage of the model will instead run on the CPU, which is slower. If possible, consider updating your model to use only operations supported by the Edge TPU. For details, visit g.co/coral/model-reqs.
Number of operations that will run on Edge TPU: 112
Number of operations that will run on CPU: 50

Operator                       Count      Status

LOGISTIC                       1          Operation is otherwise supported, but not mapped due to some unspecified limitation
DEPTHWISE_CONV_2D              14         More than one subgraph is not supported
DEPTHWISE_CONV_2D              37         Mapped to Edge TPU
QUANTIZE                       1          Mapped to Edge TPU
QUANTIZE                       4          Operation is otherwise supported, but not mapped due to some unspecified limitation
CONV_2D                        58         Mapped to Edge TPU
CONV_2D                        14         More than one subgraph is not supported
DEQUANTIZE                     1          Operation is working on an unsupported data type
DEQUANTIZE                     1          Operation is otherwise supported, but not mapped due to some unspecified limitation
CUSTOM                         1          Operation is working on an unsupported data type
ADD                            2          More than one subgraph is not supported
ADD                            10         Mapped to Edge TPU
CONCATENATION                  1          Operation is otherwise supported, but not mapped due to some unspecified limitation
CONCATENATION                  1          More than one subgraph is not supported
RESHAPE                        2          Operation is otherwise supported, but not mapped due to some unspecified limitation
RESHAPE                        6          Mapped to Edge TPU
RESHAPE                        4          More than one subgraph is not supported
PACK                           4          Tensor has unsupported rank (up to 3 innermost dimensions mapped)

The good thing about this is that edgetpu compiler does not throw any errors about incorrect input, or not quantized. If I use this model on a script on the Google Coral, which has an EdgeTPU, it runs and exits without problems, but it fails on the object detection. If with a common tensorflow model and an image it detects 2 objects, now it detects the maximum available by the script, in my case 10.